## 148: Recuperación de Contraseña con Token Temporal

Implementar un sistema para que los usuarios puedan recuperar su contraseña a través de un token temporal enviado al correo, con una expiración limitada.

🧩 1. Ampliar el modelo User con campos para recuperación
python
Copiar
Editar
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    email = Column(String, unique=True, index=True)
    hashed_password = Column(String)
    is_active = Column(Boolean, default=False)
    verification_token = Column(String, nullable=True)
    token_expiration = Column(DateTime, nullable=True)

    reset_token = Column(String, nullable=True)  # 🆕 Token para restablecer contraseña
    reset_token_expiration = Column(DateTime, nullable=True)  # 🆕 Fecha de expiración
🧩 2. Endpoint para solicitar recuperación
python
Copiar
Editar
from fastapi import BackgroundTasks

@app.post("/solicitar-recuperacion")
def solicitar_recuperacion(email: str, background_tasks: BackgroundTasks):
    user = db.query(User).filter_by(email=email).first()
    if not user:
        raise HTTPException(status_code=404, detail="Correo no registrado")

    user.reset_token = secrets.token_urlsafe(32)
    user.reset_token_expiration = datetime.utcnow() + timedelta(hours=1)
    db.commit()

    background_tasks.add_task(enviar_correo_recuperacion, user.email, user.reset_token)
    return {"mensaje": "📧 Revisa tu correo para restablecer tu contraseña"}
🧩 3. Endpoint para restablecer la contraseña
python
Copiar
Editar
from passlib.context import CryptContext
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

@app.post("/restablecer-contrasena")
def restablecer_contrasena(token: str, nueva_contrasena: str):
    user = db.query(User).filter_by(reset_token=token).first()

    if not user or user.reset_token_expiration < datetime.utcnow():
        raise HTTPException(status_code=400, detail="Token inválido o expirado")

    user.hashed_password = pwd_context.hash(nueva_contrasena)
    user.reset_token = None
    user.reset_token_expiration = None
    db.commit()

    return {"mensaje": "🔐 Contraseña restablecida exitosamente"}
🧩 4. Función para enviar correo
python
Copiar
Editar
def enviar_correo_recuperacion(destinatario: str, token: str):
    link = f"https://tusitio.com/restablecer?token={token}"
    asunto = "🔐 Recuperación de contraseña"
    cuerpo = f"Haz clic en este enlace para restablecer tu contraseña: {link}"
    # Aquí va tu lógica para enviar el correo
